In [1]:
import os
import torch
import PyPDF2
import nltk
from scipy.spatial.distance import cosine
import numpy as np
import fitz
from sklearn.neighbors import KDTree
import textdistance

C:\Users\ayush\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def pdf_extractor(path):
    pdfReader = PyPDF2.PdfFileReader(path)
    n = len(pdfReader.pages)
    print(n)
    txt = ''
    for i in range(n):
        temp = pdfReader.pages[i].extractText()
        txt = txt + temp
        
    tokens = nltk.sent_tokenize(txt.replace('\n', ''))
    #giving new name for preprocess strings
    #keeping only sentences with length greater than 3
    pre_tokens = []
    for sen in tokens:
        if len(sen.split(' ')) > 3:
            pre_tokens.append(sen)

In [12]:
directory = 'annoted_sustainability_reports'
headings = os.listdir(directory)
for head in headings:
    new_dir = os.path.join(directory,head)
    reports = os.listdir(new_dir)
    for rep in reports[0:2]:
        rep_dir = os.path.join(new_dir,rep)
        pdf_extractor(rep_dir)

142


KeyboardInterrupt: 

# Test to Extract Highlighted text from pdf

In [3]:
#testing how to extract highlighted text
doc = fitz.open(r"C:\Users\ayush\Documents\Columbia MS DS\NLP_RA\t3\annoted_sustainability_reports\Materials\Corteva_2021_Sustainability_and_ESG_Report.pdf")
# Total page in the pdf
# taking page for further processing
result = []
for page in doc:
    # list to store the co-ordinates of all highlights
    highlights = []
    # loop till we have highlight annotation in the page
    annot = page.first_annot
    while annot:
        if annot.type[0] == 8:
            all_coordinates = annot.vertices
#             print(all_coordinates)
            if len(all_coordinates) == 4:
                highlight_coord = fitz.Quad(all_coordinates).rect
                highlights.append(highlight_coord)
            else:
                all_coordinates = [all_coordinates[x:x+4] for x in range(0, len(all_coordinates), 4)]
                for i in range(0,len(all_coordinates)):
                    coord = fitz.Quad(all_coordinates[i]).rect
                    highlights.append(coord)
        annot = annot.next

    all_words = page.get_text_words()
    highlight_text = []
    if len(highlights) > 0:
        for h in highlights:
#             sentence = [w[4] for w in all_words if  fitz.Rect(w[0:4]).intersect(h)]
            sentence = []
            for w in all_words:
                if fitz.Rect(w[0:4]).intersects(h):
                    sentence.append(w[4])
            highlight_text.append(" ".join(sentence))
    if len(highlight_text) > 0:
        result.append(" ".join(highlight_text))
        
hg_text = nltk.sent_tokenize("\n".join(result))

In [4]:
hg_text

[]

# Getting text from the pdf NLTK

In [5]:
import re

#preprocessing text before sending to tokenizer
txt = ''
for page in doc:
    temp = page.get_text()
    txt = txt + temp

In [6]:
#using sentence tokenizer
#replacing \n with ' ' as . is followed by \n
tokens = nltk.sent_tokenize(txt.replace('\n',' '))

In [33]:
tokens

['CATERPILLAR’S ENVIRONMENTAL,   SOCIAL AND GOVERNANCE APPROACH 2021   SUSTAINABILITY  REPORT CATERPILLAR 2021 SUSTAINABILITY REPORT 2 EXECUTIVE SUMMARY  3 CHAIRMAN & CEO LETTER  3 SUSTAINABILITY HIGHLIGHTS  4 GOALS AND PROGRESS   6 SUSTAINABILITY AT CATERPILLAR  8 CORPORATE GOVERNANCE  11 ENERGY, FUEL ECONOMY   AND EMISSIONS  15 MATERIAL PROCUREMENT  31 REMANUFACTURING  34 PEOPLE  39 PHILANTHROPY  50 CONTENTS REPORTING ENVIRONMENTAL, SOCIAL AND   GOVERNANCE (ESG) DATA  56 PRIORITIZATION AND MANAGEMENT   OF ESG ISSUES  60 SUSTAINABILITY ACCOUNTING   STANDARDS BOARD (SASB) INDEX  61 GLOBAL REPORTING INITIATIVE  (GRI) INDEX  62 DATA ASSURANCE  68 ABOUT THIS REPORT  69 CORPORATE PROFILE  With 2021 sales and revenues of $51.0 billion, Caterpillar Inc. is the world’s leading manufacturer of construction and mining equipment, off-highway diesel and natural gas  engines, industrial gas turbines and diesel-electric locomotives.',
 'For nearly 100 years, we’ve been helping customers build a bet

# sent tokenize from spacy

In [108]:
from spacy.lang.en import English

In [113]:
nlp = English()

In [114]:
# Create the pipeline 'sentencizer' component

# Add the component to the pipeline
nlp.add_pipe('sentencizer')

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(txt.replace('\n',''))

# create list of sentence tokens
sents_list = []
for sent in doc.sents:
    sents_list.append(sent.text)
sents_list

['GLOBAL SUSTAINABILITY REPORT 2021For Abbott, sustainability is about operating responsibly to deliver long-term impact for people\xa0— shaping the future of healthcare to help more people live better, healthier lives.',
 'In this report, we detail our progress against the goals of our 2030 Sustainability Plan.',
 'The data presented reflect 2021 performance unless otherwise stated.',
 'We have aligned our reporting with the requirements of leading environmental, social and governance (ESG) ratings and sustainability indices, seeking stakeholder feedback to drive continuous improvement.',
 'WelcomeTABLE OF CONTENTS WelcomeA Message From Our Chairman and CEO 3Our Purpose in Action 4About Abbott 7Advancing Our Vision 8Our 2030 Sustainability Plan 9 Sustainability in Everything We Do 14Innovate for Access and Affordability Our Approach to Access and Affordability 16Innovating for Impact 18Innovating to Improve Health Outcomes 19Pricing Discipline to Improve Access 20Infrastructure and Tr

# Preprocessing Strings

In [7]:
#giving new name for preprocess strings
#keeping only sentences with length greater than 3

# NOTE: We are exceeding 512 tokens for some sentences which is the maximum allowed for Roberta
pre_tokens = []
for sen in tokens:
    if len(sen.split(' ')) > 3:
        pre_tokens.append(sen)

In [35]:
# for x in pre_tokens:
#     if len(x.split(' ')) > 500:
#         print(x.split(' ')[0:400])
#         print('yooo -----------------------------------------------------------------------------------------------')

# Importing the Model

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

model = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-f",output_hidden_states = True)

In [9]:
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50500, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

# Getting sentence embeddings
## Using word embeddings to generate the Sen Embeddings

In [10]:
corpus = []


for sents in pre_tokens:
    target = []
    #splitting the sentences on ' '
    for ids,sen in enumerate(sents.split(' ')):
        
        marked_text = "[CLS] " + sen + " [SEP]"
        # Split the sentence into tokens.
        tokenized_text = tokenizer.tokenize(marked_text)


        # Map the token strings to their vocabulary indeces.
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


        #creating segment ids for the sentence
        segments_ids = [1] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        hidden_states = []
        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            # Evaluating the model will return a different number of objects based on 
            # how it's  configured in the `from_pretrained` call earlier. In this case, 
            # becase we set `output_hidden_states = True`, the third item will be the 
            # hidden states from all layers. See the documentation for more details:
            # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

            hidden_states = outputs[-1]


        #print('Tensor shape for each layer: ', hidden_states[0].size())

        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)

        # Swap dimensions 0 and 1.
        token_embeddings = token_embeddings.permute(1,0,2)

        # token_embeddings.size()

        # Stores the token vectors, with shape [22 x 3,072]
        token_vecs_cat = []

        # `token_embeddings` is a [22 x 12 x 768] tensor.

        # For each token in the sentence...
        for token in token_embeddings:

            # `token` is a [12 x 768] tensor

            # Concatenate the vectors (that is, append them together) from the last 
            # four layers.
            # Each layer vector is 768 values, so `cat_vec` is length 3,072.
            sum_vec = torch.sum(token[-4:], dim=0)

            # Use `cat_vec` to represent `token`.
            token_vecs_cat.append(sum_vec)

        target.append(torch.stack(token_vecs_cat, dim = 0).mean(dim = 0))
        
    corpus.append(torch.stack(target, dim = 0).mean(dim = 0))

In [37]:
len(corpus)

660

# Keyword Embedding Mean Generation

In [11]:
keywords = ['Electric vehicle',
'Solar',
'Wind',
'Hydroelectric' ,
'Nuclear',
'REC',
'Efficiency',
'Deforestation',
'Afforestation',
'carbon',
'credit',
'capture',
'sequestration',
'storage',
'Hydrogen',
'Geothermal',
'Biomass',
'Renewable', 
'Energy',
'emissions',
'reforestation',
'Decreased', 
'Reduced']

In [12]:
target = []
for ids,sen in enumerate(keywords):
    
    marked_text = "[CLS] " + sen + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)


    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


    #creating segment ids for the sentence
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    hidden_states = []
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

        hidden_states = outputs[-1]


    #print('Tensor shape for each layer: ', hidden_states[0].size())

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # token_embeddings.size()

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(sum_vec)
        
    target.append(torch.stack(token_vecs_cat, dim = 0).mean(dim = 0))

In [13]:
#calculating mean of keywords of the word embeddings
kw_mean = torch.stack(target, dim = 0).mean(dim = 0)

# Ranking Sentences based upon KW mean

In [14]:
a = torch.stack(corpus)
all_embeddings = a
normed_embeddings = (all_embeddings.T / (all_embeddings**2).sum(axis=1) ** 0.5).T
indexer = KDTree(normed_embeddings)

In [15]:
top_10 = indexer.query(kw_mean.reshape(1, -1),return_distance = False, k = len(pre_tokens))
ranked = np.array(pre_tokens)[top_10]

In [19]:
ranked[0]

array(['GRI CONTENT INDEX      SASB INDEX      COP INDEX      TCFD INDEX     SDG INDEX       ADDITIONAL REFERENCES 2021 SASB INDEX ESG INDEXES SASB CHEMICALS SUSTAINABILITY   STANDARD (2018) CODE TOPIC METRIC PAGE OF REFERENCE/ RESPONSE  RT-CH-110a.1 Greenhouse Gas Emissions Gross global Scope 1 emissions, percentage covered under emissions-limiting  regulations GHG Emissions RT-CH-110a.2 Greenhouse Gas Emissions Discussion of long-term and short-term strategy or plan to manage Scope 1  emissions, emissions reduction targets, and an analysis of performance against  those targets Climate Strategy RT-CH-120a.1 Air Quality Air emissions of the following pollutants: (1) NOX (excluding N2O) Air Emissions RT-CH-120a.1 Air Quality Air emissions of the following pollutants: (2) SOX RT-CH-120a.1 Air Quality Air emissions of the following pollutants: (3) volatile organic compounds (“VOCs”) RT-CH-120a.1 Air Quality Air emissions of the following pollutants: (4) hazardous air pollutants (“HAPs”) R

# Calculate precision 

In [230]:
textdistance.LCSStr().similarity('ay','ayush')

2

In [26]:
if not hg_text:
    print('wow')

wow


In [20]:
result = []
pre = []
for x in hg_text:
    mx = -1
    for idx,y in enumerate(list(ranked[0])):
        score = textdistance.LCSStr().similarity(x,y)
        if score > mx:
            mx = score
            ans = idx
            ans_sen = y
        
    result.append([x,ans,ans_sen])
    pre.append(ans)
    

In [21]:
result

[]

In [52]:
with open(r"C:\Users\ayush\Downloads\ranked_results.txt", "a") as f:
    f.write('Abbott_2021_sustainability_report.pdf\n')
    for a,b,c in result:
        f.write(f"Highlighted Text:\n{a}\n Matched Sentence:\n {c}\n Index:{b}\n")
        f.write('*************************************************\n')

In [17]:
sorted(pre)

[]

In [18]:
#calculating average precision
ans = 0
for idx,ele in enumerate(pre):
    ele += 1
    idx += 1
    ans += (idx/ele)
print(f'precision is: {ans/len(pre)}')

ZeroDivisionError: division by zero

In [36]:
path = os.path.join(r"C:\Users\ayush\Downloads",'Stats.txt')
with open(path,'a') as f:
    f.write(f'{rep.split(".")[0]}\t{len(list(ranked[0]))}\t{len(pre_list)}\t{pre} \n')


In [31]:
pre_list = []
pre = 0

In [32]:
rep = 'Corteva_2021_Sustainability_and_ESG_Report.pdf'